# Using EcoFOCIpy to process raw field data

## Cruise ID - DY1707l2

## BTL Data + Nutrient Data

This is a streamlined version of generation routines to merge bottle data and Mordy Nut. Lab Nutrient Data for long term archive

In [17]:
import yaml
import glob
import pandas as pd
import os

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

In [18]:
sample_data_dir = '/Users/bell/ecoraid/2017/CTDcasts/dy1707l1/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [19]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
nutdatafile = sample_data_dir+'working/DiscreteNutrients/DY1707 Nutrient Data.csv' #<- point to cruise and process all files within
cruise_name = 'dy1707l1' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY1707L1.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDpNutsWOCE.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2017/CTDcasts/dy1707l1/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2017/CTDcasts/dy1707l1/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2017/CTDcasts/dy1707l1/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2017/CTDcasts/dy1707l1/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2017/CTDcasts/dy1707l1/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2017/CTDcasts/dy1707l1/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2017/CTDcasts/dy1707l1/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2017/CTDcasts/dy1707l1/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2017/CTDcasts/dy1707l1/rawconverted/ctd010.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [21]:
cruise_data['ctd001.btl']

,sbeox0ps,sbeox0ml/l,sbox0mm/kg,density00,density11,sbeox1ps,sbeox1ml/l,sbox1mm/kg,fleco-afl,sal00,sal11,t090c,t190c,times,prdm,par,scan,datetime
bottle,,,,,,,,,,,,,,,,,,
1.0,97.045,6.2425,272.134,1024.4762,1024.4679,97.112,6.2469,272.324,1.8187,31.8270,31.8162,10.0823,10.0817,608.250,3.181,202.42,14599.0,2017-08-22 01:12:00
2.0,96.745,6.2235,271.303,1024.4832,1024.4759,96.781,6.2258,271.406,1.8358,31.8268,31.8161,10.0805,10.0743,609.958,4.696,138.33,14640.0,2017-08-22 01:12:02


## Load csv Nutrient File

In [22]:
nut_data = pd.read_csv(nutdatafile,delimiter='\t|,')
nut_data

/var/folders/6r/n_mpcj7d7pdf9ncqvjy5vb8c0000jm/T/ipykernel_68429/1253400260.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  nut_data = pd.read_csv(nutdatafile,delimiter='\t|,')


,Cruise,Cast,Niskin,PO4 (uM),Sil (uM),NO3 (uM),NO2 (uM),NH4 (uM)
0,dy1707,2,1,1.144,14.9,9.4,0.39,0.40
1,dy1707,2,2,1.052,13.7,7.7,0.40,0.59
2,dy1707,2,3,0.937,11.8,5.0,0.37,1.00
3,dy1707,2,4,0.876,11.9,4.3,0.36,1.05
4,dy1707,2,5,0.817,10.1,3.2,0.31,0.95
...,...,...,...,...,...,...,...,...
123,dy1707,18,7,1.026,16.6,9.8,0.18,0.10
124,dy1707,18,8,0.817,14.1,6.4,0.11,0.13
125,dy1707,18,9,0.585,11.8,2.6,0.03,0.06
126,dy1707,18,10,0.583,11.8,2.6,0.03,0.07


## Merge Bottle and Nutrient Data but drop non nutrient vars?

In [23]:
keep_param = ['bottle','prdm']
# keep_param = ['bottle','prsm']
# keep_param = ['bottle','depsm']

for cast,cdata in cruise_data.items():
    try:
        matchcast = int((cast.split('.')[0]).lower().split('ctd')[-1])
        cruise_data[cast] = pd.merge(nut_data[nut_data['Cast']==matchcast],
                                      cdata.reset_index()[keep_param],
                                      right_on='bottle',
                                      left_on='Niskin').set_index('bottle').drop(columns=['Cast'])
    except:
        continue

## Add Deployment meta information

In [24]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [25]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)

## Add institutional meta-information


In [26]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [27]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            'Sil (uM)':'SI',
                            'PO4 (uM)':'PO4',
                            'NO2 (uM)':'NO2', 
                            'NO3 (uM)':'NO3',
                            'NH4 (uM)':'NH4',
                            'Sil Flag':'SI_WOCE_FLAG',
                            'PO4 Flag':'PO4_WOCE_FLAG',
                            'NO2 Flag':'NO2_WOCE_FLAG', 
                            'NO3 Flag':'NO3_WOCE_FLAG',
                            'NH4 Flag':'NH4_WOCE_FLAG',
                            'Niskin':'BTLID',
                            'prdm':'pressure',
                            'prsm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='excellent') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast.zfill(3)+'_nut.nc',format="NETCDF3_CLASSIC")
    except:
        os.remove(cruise_name+'c'+cast.zfill(3)+'_nut.nc')
        print(f'Skipping {cast}')

Skipping 001
Skipping 010


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history